In [14]:
import lseg.data as ld
import refinitiv.data as rd
import numpy as np
import pandas as pd
import configparser as cp
from datetime import datetime
from DataQuery_utils import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Simplified Equity Signals Extraction  
## Russell 3000 (AMER) and STOXX Europe 600 (EMEA)

Note that stock whose whose 3-month Average Daily Volume (ADV) is below 5M USD has not been filtered out yet.
Check the other file. Also currency and normalisation needs to be updated once we decide on signals.

In [2]:
# Open and close LSEG session
def open_session():
    try:
        ld.open_session()
        print("✓ LSEG session opened successfully")
    except Exception as e:
        print(f"✗ Error opening session: {str(e)}")
        raise

def close_session():
    try:
        ld.close_session()
        print("✓ Session closed")
    except Exception as e:
        print(f"✗ Error closing session: {str(e)}")


In [16]:
"""Get index constituents"""

def get_constituents(index_ric):
    try:
        data = ld.get_data(
            universe=index_ric,
            fields=[
                'TR.CommonName',
                'TR.CompanyMarketCap',
                'TR.HeadquartersCountry',
                'TR.GICSSector',
                'TR.GICSIndustry'
            ]
        )
        return data
    except Exception as e:
        print(f"Error fetching constituents: {str(e)}")
        return pd.DataFrame()


In [43]:
open_session()

✓ LSEG session opened successfully


In [5]:
universes = {
        'AMER_Russell3000': '0#.RUA', # Russell 3000
        'EMEA_STOXX600': '0#.STOXX'
    }

In [11]:
# Qurey useful for distinguish large or small stocks
#query = SCREEN.express.universe('0#.SPX').conditions(FORMULA('TR.CompanyMarketCap', '>20000000000'), IN(
    #'TR.TRBCEconomicSector', 'Technology')).currency('USD').query
#query

'SCREEN(U(IN(0#.RUA)),TR.AvgDailyValueTraded(60D)>5000000,CURN=USD)'

In [36]:
"""
        Get all signals for given RICs
        
        Signals include:
        - Price data
        - Momentum (1M, 3M, 6M, 12M returns)
        - Valuation (P/B, P/E, P/S, EV/EBITDA)
        - Book-to-Market ratio
        - Earnings estimates
        - Quality metrics (ROE, ROA, Debt/Equity)
        - Volatility and Beta
        - Liquidity
        """

def get_signals(rics, batch_size=50):
    fields = {
        # Identifiers
        'TR.CommonName': 'Company_Name',
        'TR.CompanyMarketCap': 'Market_Cap',
        #Price
        'TR.PriceClose': 'Price', # need to add currency?
        #Momentum signals in percentage?
        'TR.TotalReturn1Mo': 'Return_1M',
        'TR.TotalReturn3Mo': 'Return_3M',
        'TR.TotalReturn6Mo': 'Return_6M',
        'TR.TotalReturn12Mo': 'Return_12M',
        #Valuation metrics
        'TR.PriceToBookValue': 'Price_To_Book', # lower is better
        'TR.BookValuePerShare': 'Book_Value_Per_Share',
        'TR.PriceToEarnings': 'PE_Ratio', # lower = cheaper
        'TR.PriceToSales': 'PS_Ratio', # lower = cheaper
        'TR.EVToEBITDA': 'EV_To_EBITDA', # lower = cheaper
        # Earnings estimates from analysts
        'TR.EPSMeanEstimate': 'EPS_Mean_Estimate',
        'TR.EPSMedianEstimate': 'EPS_Median_Estimate',
        'TR.EPSNumEstimates': 'Num_Analysts',
        'TR.EPSGrowthRate': 'EPS_Growth_Rate',
        'TR.TargetPrice': 'Target_Price',
         # Quality metrics
        'TR.ROE': 'ROE',
        'TR.ROA': 'ROA',
        'TR.DebtToEquity': 'Debt_To_Equity',
        'TR.CurrentRatio': 'Current_Ratio',
        # Risk metrics
        'TR.Volatility30D': 'Volatility_30D', # in %
        'TR.Beta': 'Beta', # <1, less volatile
        # Liquidity
        'TR.Volume': 'Volume',
        'TR.VolumeAvg30D': 'Avg_Volume_30D',
        # Fundamental
        'TR.Revenue': 'Revenue',
        'TR.NetIncome': 'Net_Income',
        'TR.FreeCashFlow': 'Free_Cash_Flow',
        'TR.DividendYield': 'Dividend_Yield', # in %
        # Sector
        'TR.GICSSector': 'Sector',
        'TR.HeadquartersCountry': 'Country',
        
        # ============= STARMINE SIGNALS ============= to be added
            

        
    }
    
    all_data = []
    field_list = list(fields.keys())
    
    for i in range(0, len(rics), batch_size):
        batch = rics[i:i+batch_size]
        batch_num = i // batch_size + 1
        total_batches = (len(rics) - 1) // batch_size + 1
        
        print(f"  Processing batch {batch_num}/{total_batches} ({len(batch)} securities)...", end=' ')
        
        try:
            data = ld.get_data(universe=batch, fields=field_list)
            
            if not data.empty:
                data.rename(columns=fields, inplace=True)
                all_data.append(data)
                print("✓")
            else:
                print("✗ (no data)")
                
        except Exception as e:
            print(f"✗ Error: {str(e)[:50]}")
            continue
    
    if not all_data:
        return pd.DataFrame()
    
    combined = pd.concat(all_data, axis=0)
    
    # Calculate Book-to-Market ratio (inverse of P/B)
    if 'Price_To_Book' in combined.columns:
        combined['Book_To_Market'] = 1 / combined['Price_To_Book'].replace(0, np.nan)
    
    # Add upside potential (Target Price / Current Price - 1)
    if 'Target_Price' in combined.columns and 'Price' in combined.columns:
        combined['Upside_Potential'] = (combined['Target_Price'] / combined['Price'] - 1) * 100
            
    
    return combined


In [37]:
russell = get_constituents("0#.RUA")

rics = russell["Instrument"].astype(str).tolist()


signals = get_signals(rics)


  Processing batch 1/59 (50 securities)... ✓
  Processing batch 2/59 (50 securities)... ✓
  Processing batch 3/59 (50 securities)... ✓
  Processing batch 4/59 (50 securities)... ✓
  Processing batch 5/59 (50 securities)... ✓
  Processing batch 6/59 (50 securities)... ✓
  Processing batch 7/59 (50 securities)... ✓
  Processing batch 8/59 (50 securities)... ✓
  Processing batch 9/59 (50 securities)... ✓
  Processing batch 10/59 (50 securities)... ✓
  Processing batch 11/59 (50 securities)... ✓
  Processing batch 12/59 (50 securities)... ✓
  Processing batch 13/59 (50 securities)... ✓
  Processing batch 14/59 (50 securities)... ✓
  Processing batch 15/59 (50 securities)... ✓
  Processing batch 16/59 (50 securities)... ✓
  Processing batch 17/59 (50 securities)... ✓
  Processing batch 18/59 (50 securities)... ✓
  Processing batch 19/59 (50 securities)... ✓
  Processing batch 20/59 (50 securities)... ✓
  Processing batch 21/59 (50 securities)... ✓
  Processing batch 22/59 (50 securities)...

In [38]:
signals.head()

,Instrument,Company Common Name,Company Market Cap,Price Close,1 Month Total Return,3 Month Total Return,6 Month Total Return,Book Value Per Share,Enterprise Value To EBITDA (Daily Time Series Ratio),Earnings Per Share - Mean Estimate,Current Ratio,Volatility - 30 days,Volume,Revenue,Net Income Incl Extra Before Distributions,Free Cash Flow,Dividend yield,GICS Sector Name,Country of Headquarters
0,EA.OQ,Electronic Arts Inc,50050742600.0,200.0,-2.095164,-0.096318,22.949305,24.374046,37.318678,8.48109,0.94709,8.168761,1007782,7463000000,1121000000,1012000000.0,0.38,Communication Services,United States of America
1,FRD.OQ,Friedman Industries Inc,143239345.48,20.14,0.255965,-4.9834,33.286305,19.068773,12.8492,<NA>,4.34367,34.347025,8499,444600000,6085000,4259000.0,0.794439,Materials,United States of America
2,GMRE.N,Global Medical REIT Inc,491111523.54,36.63,6.824147,15.679835,17.808838,40.503367,13.490603,0.27333,<NA>,17.090784,32199,138780000,6633000,-104870000.0,11.539394,Real Estate,United States of America
3,PEN.N,Penumbra Inc,13378102063.4,341.02,7.587469,28.87646,42.097587,29.790825,67.992639,3.78547,6.00666,41.922867,193930,1194615000,14012000,16532000.0,<NA>,Health Care,United States of America
4,BMRC.OQ,Bank of Marin Bancorp,441213623.8,27.4,5.273358,8.028559,26.374062,24.755614,<NA>,2.3386,<NA>,31.455653,46607,<NA>,-35675000,<NA>,3.649635,Financials,United States of America


In [44]:
close_session()

✓ Session closed
